In [1]:
import numpy as np
import pandas as pd

In [2]:
path='https://raw.githubusercontent.com/ronishg27/ML-labs/main/naive-bayes/data.csv'

In [3]:
df = pd.read_csv(path, index_col=0)
print(df)

      Outlook Temperature Humidity    Wind Play Tennis
Day                                                   
D1      Sunny         Hot     High    Weak          No
D2      Sunny         Hot     High  Strong          No
D3   Overcast         Hot     High    Weak         Yes
D4       Rain        Mild     High    Weak         Yes
D5       Rain        Cool   Normal    Weak         Yes
D6       Rain        Cool   Normal  Strong          No
D7   Overcast        Cool   Normal  Strong         Yes
D8      Sunny        Mild     High    Weak          No
D9      Sunny        Cool   Normal    Weak         Yes
D10      Rain        Mild   Normal    Weak         Yes
D11     Sunny        Mild   Normal  Strong         Yes
D12  Overcast        Mild     High  Strong         Yes
D13  Overcast         Hot   Normal    Weak         Yes
D14      Rain        Mild     High  Strong          No


In [4]:
X, y=df.drop(["Play Tennis"],axis=1) ,df["Play Tennis"]

In [5]:
def accuracy_score(y_true, y_pred):
	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100 ,2)

In [6]:
class  NaiveBayes:

	def __init__(self):
		self.features = list
		self.likelihoods = {}
		self.class_priors = {}
		self.pred_priors = {}

		self.X_train = np.array
		self.y_train = np.array
		self.train_size = int
		self.num_feats = int

	def fit(self, X, y):

		self.features = list(X.columns)
		self.X_train = X
		self.y_train = y
		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]

		for feature in self.features:
			self.likelihoods[feature] = {}
			self.pred_priors[feature] = {}

			for feat_val in np.unique(self.X_train[feature]):
				self.pred_priors[feature].update({feat_val: 0})

				for outcome in np.unique(self.y_train):
					self.likelihoods[feature].update({feat_val+'_'+outcome:0})
					self.class_priors.update({outcome: 0})

		self._calc_class_prior()
		self._calc_likelihoods()
		self._calc_predictor_prior()

	def _calc_class_prior(self):

		""" P(c) - Prior Class Probability """

		for outcome in np.unique(self.y_train):
			outcome_count = sum(self.y_train == outcome)
			self.class_priors[outcome] = outcome_count / self.train_size

	def _calc_likelihoods(self):

		""" P(x|c) - Likelihood """

		for feature in self.features:

			for outcome in np.unique(self.y_train):
				outcome_count = sum(self.y_train == outcome)
				feat_likelihood = self.X_train[feature][self.y_train[self.y_train == outcome].index.values.tolist()].value_counts().to_dict()

				for feat_val, count in feat_likelihood.items():
					self.likelihoods[feature][feat_val + '_' + outcome] = count/outcome_count


	def _calc_predictor_prior(self):

		""" P(x) - Evidence """

		for feature in self.features:
			feat_vals = self.X_train[feature].value_counts().to_dict()

			for feat_val, count in feat_vals.items():
				self.pred_priors[feature][feat_val] = count/self.train_size


	def predict(self, X):

		""" Calculates Posterior probability P(c|x) """

		results = []
		X = np.array(X)

		for query in X:
			probs_outcome = {}
			for outcome in np.unique(self.y_train):
				prior = self.class_priors[outcome]
				likelihood = 1
				evidence = 1

				for feat, feat_val in zip(self.features, query):
					likelihood *= self.likelihoods[feat][feat_val + '_' + outcome]
					evidence *= self.pred_priors[feat][feat_val]
				posterior = (likelihood * prior)

				probs_outcome[outcome] = posterior

			result = max(probs_outcome, key = lambda x: probs_outcome[x])
			results.append(result)

		return np.array(results)


In [7]:
nb_clf = NaiveBayes()
nb_clf.fit(X, y)

print("Train Accuracy: {}".format(accuracy_score(y, nb_clf.predict(X))))

Train Accuracy: 92.86


In [8]:
query = np.array([['Sunny','Cool','Normal','Strong']])
print("Query:- {} ---> {}".format(query, nb_clf.predict(query)))

Query:- [['Sunny' 'Cool' 'Normal' 'Strong']] ---> ['Yes']
